# 14. 분산형 공유 변수
- 브로드캐스트 변수와 어큐뮬레이터라는 두개의 타입이 존재
  - **어큐뮬레이터** : 모든 태스크의 데이터를 공유 결과에 추가할 수 있음
  - **브로드캐스트** : 모든 워커 노드에 큰 값을 저장하므로 재전송없이 많은 스파크 액션에서 재사용할 수 있음

### 14.1 브로드캐스트 변수
- 변하지 않는 값(불변성 값)을 클로저 함수의 변수로 캡슐화하지 않고 클러스터에 효율적으로 공유
- 브로드캐스트 변수는 모든 태스크마다 직렬화하지 않고 클러스터의 모든 머신에 캐시하는 불변성 공유 변수

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
  .builder \
  .appName("Spark Distrubuted Shared Variable example") \
  .getOrCreate()

In [4]:
my_collection = "Spark The Definitive Guide : Big Data Processing Made Sample".split(" ")
words = spark.sparkContext.parallelize(my_collection, 2)
words.collect()

Out[7]: ['Spark',
 'The',
 'Definitive',
 'Guide',
 ':',
 'Big',
 'Data',
 'Processing',
 'Made',
 'Sample']

In [5]:
supplementalData = {"Spark": 1000, "Definitive" : 200, "Big": -300, "Simple" : 100} # Dict

- value 메서드를 사용해 값 참조
- 직렬화된 함수에서 브로드캐스트된 데이터를 직렬화하지 않아도 접근 가능
  - 직렬화(자바) : [http://woowabros.github.io/experience/2017/10/17/java-serialize.html](http://woowabros.github.io/experience/2017/10/17/java-serialize.html)
  - 직렬화(스파크) : [https://12bme.tistory.com/436](https://12bme.tistory.com/436)
- 브로드캐스트 변수는 모든 태스크마다 직렬화하지 않고 클러스터의 모든 머신에 캐시하는 불변성 공유 변수
  - 직렬화와 역직렬화에 대한 부하를 크게 줄일 수 있음
  - 큰 크기의 데이터를 사용하는 경우 효과가 커짐
  - UDF나 Dataset에서도 사용할 수 있으며 동일한 효과

In [7]:
suppBroadcast = spark.sparkContext.broadcast(supplementalData)
suppBroadcast.value

Out[18]: {'Spark': 1000, 'Definitive': 200, 'Big': -300, 'Simple': 100}

In [8]:
# suppBroadcast.value.get(word, 0) : dict의 get함수 사용(있으면 word, 없으면 기본값 0)
# sortBy(lambda wordPair: wordPair[1]) 튜플의 2번째 값 기준으로 정렬
words.map(lambda word: (word, suppBroadcast.value.get(word, 0))) \
  .sortBy(lambda wordPair: wordPair[1]).collect()

Out[20]: [('Big', -300),
 ('The', 0),
 ('Guide', 0),
 (':', 0),
 ('Data', 0),
 ('Processing', 0),
 ('Made', 0),
 ('Sample', 0),
 ('Definitive', 200),
 ('Spark', 1000)]

### 14.2 어큐뮬레이터
- 트랜스포메이션 내부의 다양한 값을 갱신하는데 사용됨
  - 디버깅용이나 저수준 집계 생성용으로 사용 가능
- 어큐뮬레이터 값은 액션을 처리하는 과정에서만 갱신됨
  - 각 태스크에서 어큐뮬레이터를 한 번만 갱신하도록 제어
  - 따라서 재시작한 태스크는 어큐뮬레이터 값을 갱신할 수 있음
- 스파크의 지연 연산 모델에 영향을 주지 않음
- 이름이 지정된 어큐뮬레이터의 실행 결과는 스파크 UI에 표현됨

In [10]:
dbutils.fs.ls("/databricks-datasets/definitive-guide/data/flight-data/parquet/")

Out[24]: [FileInfo(path='dbfs:/databricks-datasets/definitive-guide/data/flight-data/parquet/2010-summary.parquet/', name='2010-summary.parquet/', size=0)]

In [11]:
flights = spark.read.format("parquet").option("inferSchema", "true").load("/databricks-datasets/definitive-guide/data/flight-data/parquet/*.parquet")
display(flights.limit(10))

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,1
United States,Ireland,264
United States,India,69
Egypt,United States,24
Equatorial Guinea,United States,1
United States,Singapore,25
United States,Grenada,54
Costa Rica,United States,477
Senegal,United States,29
United States,Marshall Islands,44


In [12]:
# no named accumulator
accChina = spark.sparkContext.accumulator(0)

# make name
# spark.sparkContext.register(accChina, "China")

In [13]:
def accChinaFunc(flight_row):
  destination = flight_row["DEST_COUNTRY_NAME"]
  origin = flight_row["ORIGIN_COUNTRY_NAME"]
  if "China" in (destination, origin):
    accChina.add(flight_row["count"])

- foreach는 액션이며, 액션에서만 어큐뮬레이터의 실행을 보장

In [15]:
flights.foreach(lambda flight_row: accChinaFunc(flight_row))

In [16]:
accChina.value

Out[32]: 953

In [17]:
# 증명
from pyspark.sql.functions import col, column

flights.where(col("DEST_COUNTRY_NAME") == 'China').show()
flights.where(column("ORIGIN_COUNTRY_NAME") == "China").show()

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 China| United States| 448|
+-----------------+-------------------+-----+

+-----------------+-------------------+-----+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
 United States| China| 505|
+-----------------+-------------------+-----+

### 14.2.2 사용자 정의 어큐뮬레이터
- 어큐뮬레이터를 직접 정의하려면 AccmulatorV2 클래스를 상속 받아야 함
  - 파이썬은 AccumulatorParam을 상속받아야 함

In [19]:
from pyspark.accumulators import AccumulatorParam

In [20]:
class EvenAccumulator(AccumulatorParam):
  def __init__(self, param=0):
    self.num = param
    
  def reset(self):
    self.num = 0
    
  def add(self, intvalue):
    if(intvalue%2 == 0):
      self.num += intvalue
      
  def value(self):
    return self.num
  
  def iszero(self):
    return self.num == 0

In [21]:
newAcc = EvenAccumulator()
newAcc.add(3)
newAcc.add(2)
newAcc.value()

Out[36]: 2

In [22]:
newAcc.reset()
newAcc.value()

Out[37]: 0

In [23]:
flights.foreach(lambda flight_row: newAcc.add(flight_row['count']))
newAcc.value()

Out[38]: 0

In [24]:
for i in flights:
  print(i)

Column<b'DEST_COUNTRY_NAME'>
Column<b'ORIGIN_COUNTRY_NAME'>
Column<b'count'>